In [1]:
import numpy
import vtk
from vtk.util import numpy_support
import os

In [2]:
#get reader to load tiff file
reader = vtk.vtkTIFFReader()
reader.SetFileName(r'C:\Users\miao1\Data\focused-fast-scans\trimmed_stacks\Trimmed_cell3_Iter_0_ch0_stack0000_3nm_0000000msec_0009286310msecAbs.tif')
reader.Update()

extent = reader.GetDataExtent()
dims = [extent[1]-extent[0]+1, extent[3]-extent[2]+1, extent[5]-extent[4]+1]
print('dims', dims)

pointData = reader.GetOutput().GetPointData()
arrayData = pointData.GetArray(0)
vol = numpy_support.vtk_to_numpy(arrayData)
vol = vol.reshape(dims, order='F')

dims [259, 279, 289]


In [23]:
#let's first generate a binary volume based on a threshold th

th = 600
threshold = vtk.vtkImageThreshold()
threshold.SetInputConnection(reader.GetOutputPort())
threshold.ThresholdByLower(th)
threshold.ReplaceInOn()
threshold.SetInValue(0)  # values below th -> 0
threshold.ReplaceOutOn()
threshold.SetOutValue(1)  # values above th -> 0
threshold.Update()

In [24]:
%%time
mc = vtk.vtkDiscreteMarchingCubes()
mc.SetInputConnection(threshold.GetOutputPort())
mc.GenerateValues(1, 1, 1)
mc.Update()

Wall time: 475 ms


In [26]:
#We only want the largest connected component
connectivityFilter = vtk.vtkPolyDataConnectivityFilter()
connectivityFilter.SetInputData(mc.GetOutput())
connectivityFilter.SetExtractionModeToLargestRegion()
connectivityFilter.Update()

In [42]:
%%time
gSmoothingSteps = 30
filter = vtk.vtkWindowedSincPolyDataFilter()

if(gSmoothingSteps > 0):
    print("smoothing...")
    filter.SetNumberOfIterations(gSmoothingSteps)
    filter.SetPassBand(0.1)
    filter.SetInputData(connectivityFilter.GetOutput())
    filter.Update()


smoothing...
Wall time: 1.27 s


In [41]:
writer = vtk.vtkSTLWriter()
writer.SetInputConnection(filter.GetOutputPort())
writer.SetFileTypeToBinary()
writer.SetFileName("cell_{}.stl".format(th))
writer.Write()

1

In [44]:
properties = vtk.vtkMassProperties()
properties.SetInputConnection(filter.GetOutputPort())
print("surface area", properties.GetSurfaceArea())
print("volume", properties.GetVolume())

surface area 110634.10165558863
volume 940194.2268488472
